![Podmanlogo](Pictures/podman-logo.png)

# Running containers at system start

There are many situations in which you may want to start a container automatically when the system starts. A good example of this would be an edge environment, think of a small ruggered computer in a factory that runs containerized software using Podman. If there is a power failure the system will of course shut down, once the power is back the system will boot on and most likely you would like your containers to start automatically after boot. We can easily achieve this by taking advantage of systemd and Podman.

In most linux distributions systemd is the parent of all processes on the system, it is executed by the kernel and is responsible for starting all other processes. If we want to automatically start a process, like a container, at system start it's a best practice to take advantage of systemd. Podman is capable of helping with this process, lets see how.

We will start deploying a container with Podman the same way we've been doing it during the rest of the workshop.

In [ ]:
!podman run --rm -d --name my-autostart-container fedora sleep 99999

Check the container is properly deployed

In [ ]:
!podman ps -a

Our container is running and it's seen by the operating system as a process. Now we would need to generate a systemd unit file that systemd can use to execute this process when the system starts. Podman provides a very easy way of generating this file from a running container:

In [ ]:
!podman generate systemd --new --files --name my-autostart-container

As you can see the output explains that this is a deprecated command, we will come back to that in a minute. First lets take a look at the output file that was generated.

In [ ]:
!cat container-my-autostart-container.service

As you can see Podman has generated the file for us, the only additional thing we would need to do now is to place this file in ls "$HOME/.config/systemd/user" and reload systemd running "systemctl --user daemon-reload". Once you've done that your system will automatically start your container on boot.

This is cool but, as you've seen, the generated file is long and complex to read specially for non experienced users. Also, most of the times we work with containers we want to work in a declarative manner. This means we would like to have a file in which we define how our application should run and an orchestrator will make it work, this is the default way of working with kubernetes and has become a de-facto standard for containerized environments.

For these reasons Podman introduced Quadlets in version 4.4. This technology brings the capability of writting a simple and easy to read file that can be used how you want you container to run (following the declarative model). Lets see how it works.

The only thing you need to do to take advantage of Quadlets is creating a symple file with the definition of your container and with the name ending in ".container":

In [ ]:
cat << EOF > my-quadlet.container
[Unit]
Description=The sleep container
After=local-fs.target

[Container]
Image=docker.io/library/fedora:latest
Exec=sleep 1000

[Install]
# Start by default on boot
WantedBy=multi-user.target default.target
EOF
cat my-quadlet.container


As you can see this is a very easy to read file that allows you to take advantage of the declarative way of working. Once you have created this file you just need to move it to $HOME/.config/containers/systemd/ if you're a rootless user or to /usr/share/containers/systemd/ if you have root access to the system. Once you've done that reload systemd by running "systemctl --user daemon-reload". Let's do it.

In [ ]:
#mkdir ~/.config/containers/systemd/ -p
!mv ./my-quadlet.container ~/.config/containers/systemd/
!systemctl --user daemon-reload

Now we can use standard linux commands to check the status of our workload:

In [ ]:
!systemctl --user status my-quadlet.container

But that's not all!!! The best part of Quadlets is that it can take kubernetes yaml files as unit files, making it super easy to transition from Podman to kubernetes and the other way around. If you know the basics of kubernetes you may be thinking that's not possible as kubernetes works with pods as the minimum schedulable unit while Podman works with containers, but Podman can also work with pods. We will explain how to manage pods with Podman in next section.